In [ ]:
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")

# Lecture 4: Steepest descent and Newton's method

## Let us define the same function as on the previous lecture for testing

In [ ]:
def f_simple(x):
    return (x[0] - 10.0)**2 + (x[1] + 5.0)**2+x[0]**2

## Gradient-based optimization

If gradients are available, they should always be used --> improve convergence

Gradient-based methods usually converge to a local minimum which is nearest to the starting point (**Why?**)

If analytical formulas of the gradients are not available, one way is to numerically approximate them by using finite differences ($ℎ>0$)
* forward difference: $\frac{f(x+h)-f(x)}{h}=\nabla f(x)+O(h)$
* central difference: $\frac{f(x+h)-f(x-h)}{2h}=\nabla f(x)+O(h^2)$
* central difference is more accurate but requires twice as many function evaluations (in $𝑥+ℎ$ and $𝑥−ℎ$)

## Automatic differentiation in Python

An alternative way is to use automatic differentiation

https://en.wikipedia.org/wiki/Automatic_differentiation

Import automatic differentiation package for Python

Needs to be installed typing
```
pip install ad
```

You can ask for gradient and hessian using the <pre>ad.gh</pre> function.

 Let us use automatic differentiation for the function <it>f</it>  that we defined.

In [ ]:
import ad
grad_f, hess_f = ad.gh(f_simple)

In [ ]:
print("At the point (1,2) gradient is ", grad_f([1,2]), " and hessian is ",hess_f([1,2]))

### Let us visualize the gradient

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from pylab import meshgrid
def visualize_gradient(f,point,x_lim,y_lim):
    grad_point = np.array(ad.gh(f)[0](point)) # computes the gradient
    grad_point = 10.0*grad_point/np.linalg.norm(grad_point) # scale the gradient vector so that it is easier to see
    X,Y,Z = point[0],point[1],f(point) # point that we are interested in and the function value in that point 
    U,V,W = grad_point[0],grad_point[1],0 # gradient vector, last component is zero since the gradient has 2 components
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d') 
    x = np.arange(x_lim[0],x_lim[1],0.1) # divide interval for x
    y = np.arange(y_lim[0],y_lim[1],0.1) # divide interval for y
    X2,Y2 = meshgrid(x, y) # make a grid for x and y
    Z2 = [f([x,y]) for (x,y) in zip (X2,Y2)] # evaluation of the function on the grid
    Z2 = np.asarray(Z2)
    surf = ax.plot_surface(X2, Y2, Z2,alpha=0.5) # surface plot for the function values
    ax.quiver(X,Y,Z,U,V,W,color='red',linewidth=2.5) # plots a 2d arrow in a 3d figure
    return plt
    

In [ ]:
visualize_gradient(f_simple,[1,-2],[0,10],[-10,0]).show()

With the lambda function we can easily visualize gradients of various functions

In [ ]:
import math 
visualize_gradient(lambda x:3*x[0]+x[1],[1,1],[0,2],[0,2]).show()

## Base algorithm for the steepest descent and Newton's algorithms
**Input:** function $f$ to be optimized, starting point $x_0$, step length rule $alpha$, stopping rule $stop$  
**Output:** A solution $x^*$ that is close to a locally optimal solution
```
set f_old as a big number and f_new as f(x0)
while a stopping criterion has not been met:
    f_old = f_new
    determine search direction d_h according to the method
    determine the step length alpha
    set x = x + alpha *d_h
    f_new = f(x)
return x
```

The way to determine search direction distinguishes steepest descent algorithm and the Newton algorithm. Different stopping rules and step sizes can be mixed and matched with both algorithms.

## Steepest Descent algorithm for unconstrained optimization

Also known as *gradient descent* or *gradient method*

In the steepest descent algorithm, the search direction is determined by the negative of the gradient $-\nabla f(x)$ which (locally) gives the direction of the steepest descent of $f$ at point $x$.

### Code in Python

Let us use a simple stopping rule, where we stop when the change is not bigger than precision and we have a fixed step size.

In [ ]:
import numpy as np
import ad
def steepest_descent(f,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
    #while np.linalg.norm(ad.gh(f)[0](x))>precision: # an alternative stopping rule
        f_old = f_new # store value at the current point
        d = -np.array(ad.gh(f)[0](x)) # search direction
        x = x+d*step # take a step
        f_new = f(x) # compute function value at the new point
        steps.append(list(x)) # save step
    return x,f_new,steps

### Solve the problem using the Python function

In [ ]:
start = [2.0,-10.0]
step_size = 0.1
precision = 0.01
(x_value,f_value,steps) = steepest_descent(f_simple,start,step_size,precision)
print("Optimal solution is ",x_value)
print(len(steps))
print(steps)

Plot the steps of solving

In [ ]:
import matplotlib.pyplot as plt

def plot_2d_steps(steps,start):
    myvec = np.array([start]+steps).transpose()
    #print(myvec)
    plt.plot(myvec[0,],myvec[1,],'ro')
    for label,x,y in zip([str(i) for i in range(len(steps)+1)],myvec[0,],myvec[1,]):
        plt.annotate(label,xy = (x, y))
    return plt

In [ ]:
plot_2d_steps(steps,start).show()

## Other stopping rules
* change in function values: $|f_{new}-f_{old}| \leq $precision 

This one we already used. Others?

* norm of the gradient: $||\nabla f(x_{new})|| \leq $precision
* change in variable values: $||x_{new}-x_{old}|| \leq $precision
* maximum number of iterations


In [ ]:
# Another example
def f_simple2(x):
    return (x[0] - 2.0)**4 + (x[0] - 2.0*x[1])**2
# Optimal solution is (2,1)

In [ ]:
start = [0.0,3.0]
step_size = .3
precision = 0.001
(x_value,f_value,steps) = steepest_descent(f_simple2,start,step_size,precision)
print("Optimal solution is ",x_value)
print(len(steps))
#print(steps)

In [ ]:
plot_2d_steps(steps,start).show()

Compare steepest descent with optimized step size:
![](images/steepest_descent.png)
*From Miettinen: Nonlinear optimization, 2007 (in Finnish)*

## Newton's method

Based on setting the search direction as $d^h=-[H(x^h)]^{-1}\nabla f(x^h).\hspace{2.0cm}$   --> **uses second derivatives!**

In one-dimensional case by the Taylor series:
$$f(x+\Delta x)\approx f(x)+f'(x)\Delta x+\frac12f''(x)\Delta x^2$$


We want to find $x$ such that $f(x)$ is at minimum and, thus, we seek to solve the equation that sets the derivative of this expression with respect to $\Delta x$ equal to zero:

$$ 0 = \frac{d}{d\Delta x} \left(f(x^h)+f'(x^h)\Delta x+\frac 1 2 f''(x^h) \Delta x^2\right) = f'(x^h)+f'' (x^h) \Delta x.$$

The solution of the above equation is $\Delta x=-f'(x^h)/f''(x^h)$. Thus, the best approximation of $x^{h+1}$ as the minimum is $x^h-f''(x^h)^{-1}f'(x^h)$.


In [ ]:
def newton(f,start,step,precision):
    f_old = float('Inf')
    x = np.array(start)
    steps = []
    f_new = f(x)
    while abs(f_old-f_new)>precision:
        f_old = f_new # store value of f at the current point
        H_inv = np.linalg.inv(np.matrix(ad.gh(f)[1](x))) # compute the inverse of the Hessian
        d = (-H_inv*(np.matrix(ad.gh(f)[0](x)).transpose())).transpose() # compute search direction
        #Change the type from np.matrix to np.array so that we can use it in our function
        x = np.array(x+d*step)[0] # take the step
        f_new = f(x) # compute function value at the new point
        steps.append(list(x)) # store step
    return x,f_new,steps

In [ ]:
start = [2.0,-10.0]
step_size = 0.1
precision = 0.001
(x_value,f_value,steps) = newton(f_simple,start,step_size,precision)
print("Optimal solution is ",x_value)
print(len(steps))

In [ ]:
plot_2d_steps(steps,start).show()

## Another example

In [ ]:
start = [0.0,3.0]
step_size = 1.0
precision = 0.001
(x_value,f_value,steps) = newton(f_simple2,start,step_size,precision)
print("Optimal solution is ",x_value)
print(len(steps))

In [ ]:
plot_2d_steps(steps,start).show()

Compare Newton's method with step size = 1.0:
![](images/newton.png)
*From Miettinen: Nonlinear optimization, 2007 (in Finnish)*

# Summary
Steepest descent
* Descent direction (does the search direction guarantee decrease in $f$): yes
* Global convergence (converges from any starting point): yes
* Local convergence: zig-zag near optimum
* Computational bottle neck: step length
* Memory consumption: $𝑂(𝑛)$

Newton’s method
* Descent direction: only if $𝐻(𝑥)^{−1}$ positive definite
* Global convergence: no
* Local convergence: yes, good if 𝑓 quadratic
* Computational bottle neck: $𝐻(𝑥 )^{−1}$ --> can be relaxed by using approximations of $H(x)^{-1}$ --> quasi-Newton
* Memory consumption: $𝑂(𝑛^2)$ --> can be reduced e.g. by using conjugate gradient methods